In [154]:
import pandas as pd

In [155]:
up_keywords = []
with open("看漲-篩掉負號.TXT",'r',encoding = 'UTF8') as file:
    for data in file.readlines():
        data = data.strip()
        up_keywords.append(data)

In [156]:
down_keywords = []
with open("看跌-篩掉負號.TXT",'r',encoding = 'UTF8') as file:
    for data in file.readlines():
        data = data.strip()
        down_keywords.append(data)

In [188]:
word_vec = up_keywords[:500] + down_keywords[:500]

In [189]:
size = len(word_vec)

In [190]:
size

1000

In [191]:
up_news = pd.read_csv('看漲-幅度大於2%.csv')
down_news = pd.read_csv('看跌-幅度大於2%.csv')

In [192]:
up_news_content = list(up_news['content'])
down_news_content = list(down_news['content'])

In [193]:
news = up_news_content + down_news_content

In [194]:
len(news)

3183

In [195]:
import re

In [196]:
def preprocess(sent): # 不切 ngram
    sent = re.sub(r'[^\w]',"",sent)
    sent = re.sub(r'[A-Za-z0-9]',"",sent)
    return sent

In [198]:
print(len(up_news), len(down_news))
print(len(up_news) + len(down_news))

1692 1491
3183


In [199]:
news_data_binary = []
news_data_times = []
for index, row in up_news.iterrows():
    text_vec = []
    content = preprocess(row["title"] + row["content"])
    
    data_bi = {
        'index': index,
        'title': row['title'],
        'content': row['content'],
        'y': 1
    }
    
    data_times = {
        'index': index,
        'title': row['title'],
        'content': row['content'],
        'y': 1
    }
    
    for word in word_vec:
        data_bi[word] = 0
        data_times[word] = 0
        if word in content:
            data_times[word] = content.count(word)
            data_bi[word] = 1
    
    news_data_binary.append(data_bi)
    news_data_times.append(data_times)

In [200]:
print(len(news_data_binary), len(news_data_times))

1692 1692


In [201]:
up_size = len(up_news)
up_size

1692

In [202]:
for index, row in down_news.iterrows():
    text_vec = []
    content = preprocess(row["title"] + row["content"])
    i = index + up_size
    
    data_bi = {
        'index': i,
        'title': row["title"],
        'content': row['content'],
        'y': -1
    }
    
    data_times = {
        'index': i,
        'title': row["title"],
        'content': row['content'],
        'y': -1
    }
    
    for word in word_vec:
        data_bi[word] = 0
        data_times[word] = 0
        if word in content:
            data_times[word] = content.count(word)
            data_bi[word] = 1
    
    news_data_binary.append(data_bi)
    news_data_times.append(data_times)

In [203]:
print(len(news_data_binary), len(news_data_times))

3183 3183


In [204]:
binary_pd = pd.DataFrame(news_data_binary)

In [207]:
base_data = pd.concat([
    binary_pd['index'],
    binary_pd['title'],
    binary_pd['content'],
    binary_pd['y']
],axis=1)

In [208]:
base_data.to_csv(r'基本資料-開根號500.csv', index = False)

In [209]:
n = 0 - size
X = binary_pd[binary_pd.columns[n:]]

In [210]:
X.to_csv(r'training_X_binary-開根號500.csv', index = False)

In [211]:
binary_pd['y'].to_csv(r'training_Y-開根號500.csv', index = False)

In [212]:
times_pd = pd.DataFrame(news_data_times)

In [213]:
times_pd.loc[1]

index                                                      1
title                             《外資》外資挺大立光：蘋果扶植Kantatsu非入股
content    受到台股重挫拖累，大立光 (3008) 今天早盤股價跌幅逾5.5%，針對市場再將去年第4季已...
y                                                          1
不適用                                                        0
                                 ...                        
類股全面                                                       0
大指數漲跌互                                                     0
利空消息                                                       0
擴大至                                                        0
連紅台股今                                                      0
Name: 1, Length: 1004, dtype: object

In [214]:
X = times_pd[times_pd.columns[n:]]

In [215]:
X.to_csv(r'training_X_times-開根號500.csv', index = False)